##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to Multi-Armed Bandits

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/intro_bandit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

## 仅保存权重值。通常在训练模型时使用。

多臂老虎机 (MAB) 是一种机器学习框架，其中的代理必须选择操作（臂）以最大化其长期累积奖励。在每个轮次中，代理都会收到有关当前状态（上下文）的一些信息，随后，它会根据此信息和在之前轮次中收集到的经验来选择一个操作。在每个轮次结束时，代理会收到与所选操作相关的奖励。

或许，最纯粹的示例就是把它的名字借给了 MAB 的问题：想象一下，我们面对着 `k` 台老虎机（单臂老虎机），我们需要找出哪一台具有最高的赔付，同时又不会损失太多金钱。

![Multi-Armed Bandits](https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Las_Vegas_slot_machines.jpg/320px-Las_Vegas_slot_machines.jpg)

每台机器尝试一次，然后选择赔付最高的机器并不是一种良好的策略：代理可能会选择一台刚开始的结果很幸运但总体上不太理想的机器。相反，代理应反复回来选择看起来不那么好的机器，以便收集有关它们的更多信息。这是多臂老虎机中的主要挑战：代理必须在利用先验知识和探索之间找到正确的组合，以避免忽略最佳操作。

每次学习器做出决定时，更实际的 MAB 实例都会涉及一些附加信息。我们将此附加信息称为“上下文”或“观测值”。


## Multi-Armed Bandits and Reinforcement Learning

Why is there a MAB Suite in the TF-Agents library? What is the connection between RL and MAB? Multi-Armed Bandits can be thought of as a special case of Reinforcement Learning. To quote [Intro to RL](https://www.tensorflow.org/agents/tutorials/0_intro_rl):

*At each time step, the agent takes an action on the environment based on its policy $\pi(a_t|s_t)$, where $s_t$ is the current observation from the environment, and receives a reward $r_{t+1}$ and the next observation $s_{t+1}$ from the environment. The goal is to improve the policy so as to maximize the sum of rewards (return).*

In the general RL case, the next observation $s_{t+1}$ depends on the previous state $s_t$ and the action $a_t$ taken by the policy. This last part is what separates MAB from RL: in MAB, the next state, which is the observation, does not depend on the action chosen by the agent.

这种相似性使我们可以重用 TF-Agents 中存在的所有概念。

- 一个输出观测值并用奖励来响应操作的**环境**。
- A **policy** outputs an action based on an observation, and
- An **agent** repeatedly updates the policy based on previous observation-action-reward tuples.


## The Mushroom Environment

For illustrative purposes, we use a toy example called the "Mushroom Environment". The mushroom dataset ([Schlimmer, 1981](https://archive.ics.uci.edu/ml/datasets/Mushroom)) consists of labeled examples of edible and poisonous mushrooms. Features include shapes, colors, sizes of different parts of the mushroom, as well as odor and many more.

![mushroom](https://archive.ics.uci.edu/ml/assets/MLimages/Large73.jpg)

与所有监督学习数据集一样，蘑菇数据集也可以转换为上下文 MAB 问题。我们使用 [Riquelme 等人（2018 年）](https://arxiv.org/pdf/1802.09127.pdf)使用的方法。在此转换中，代理接收蘑菇的特征，决定是否吃下。吃下食用蘑菇获得 +5 的奖励，而吃下毒蘑菇则会以相等的概率获得 +5 或 -35。不吃蘑菇会导致 0 奖励，而与蘑菇的类型无关。下表汇总了奖励分配：

> ```
>
> ```

```
       | edible | poisonous
```

-----------|--------|---------- 吃下  |     +5 | -35 / +5 不吃 |      0 |        0

```

```

## The LinUCB Agent

在给定观测值的情况下，要在上下文老虎机环境中获得出色的表现，需要很好地评估每个操作的奖励函数。一种可能性是使用线性函数评估奖励函数。也就是说，对于每个操作 $i$，我们都尝试找到参数 $\theta_i\in\mathbb R^d$，使估算值

$r_{t, i} \sim \langle v_t, \theta_i\rangle$

are as close to the reality as possible. Here $v_t\in\mathbb R^d$ is the context received at time step $t$. Then, if the agent is very confident in its estimates, it can choose $\arg\max_{1, ..., K}\langle v_t, \theta_k\rangle$ to get the highest expected reward.

As explained above, simply choosing the arm with the best estimated reward does not lead to a good strategy. There are many different ways to mix exploitation and exploration in linear estimator agents, and one of the most famous is the Linear Upper Confidence Bound (LinUCB) algorithm (see e.g. [Li et al. 2010](https://arxiv.org/abs/1003.0146)). LinUCB has two main building blocks (with some details omitted):

1. It maintains estimates for the parameters of every arm with Linear Least Squares: $\hat\theta_i\sim X^+_i r_i$, where $X_i$ and $r_i$ are the stacked contexts and rewards of rounds where arm $i$ was chosen, and $()^+$ is the pseudo inverse.
2. It maintains *confidence ellipsoids* defined by the inverse covariance $X_i^\top X_i$ for the above estimates.

LinUCB 的主要理念是“不确定行为优先探索”。代理通过将估计值增加与这些估计值的方差相对应的量来纳入探索。这就是置信椭圆生效的位置：对于每个臂，乐观估计值为 $\hat r_i = \max_{\theta\in E_i}\langle v_t，其中 $E_i$ 是绕 $\hat\theta_i$ 的椭圆。代理选择看起来最佳的臂 $\arg\max_i\hat r_i$。

Of course the above description is just an intuitive but superficial summary of what LinUCB does. An implementation can be found in our codebase [here](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/lin_ucb_agent.py)

## What's Next?
If you want to have a more detailed tutorial on our Bandits library take a look at our [tutorial for Bandits](https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/bandits_tutorial.ipynb). If, instead, you would like to start exploring our library right away, you can find it [here](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits). If you are even more eager to start training, look at some of our end-to-end examples [here](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2), including the above described mushroom environment with LinUCB [here](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2/train_eval_mushroom.py). 